In [1]:
import pandas as pd

In [ ]:
#ignore the first row. use second row as header
df1 = pd.read_csv('Annex C.csv', skiprows=2)
df2 = pd.read_csv('Annex B.csv', skiprows=2)
#join on 'Test ID'
df = pd.merge(df1, df2, on='Test ID')
#print row lengths of each
print(len(df1), len(df2), len(df))


In [48]:
#print table of PERIOD_DATE and test count, sorted by PERIOD_DATE = 215 days
date_counts = df['PERIOD_DATE'].value_counts()
date_counts.index = pd.to_datetime(date_counts.index, format='%d/%m/%Y')
time_span_days = date_counts.index.max() - date_counts.index.min()
time_span_days = time_span_days.days

In [45]:
df['SLOT_TIME'] = pd.to_datetime(df['SLOT_TIME'], format='%H:%M')
df['TIME_BIN'] = df['SLOT_TIME'].dt.floor('60T')
df['TIME_BIN_STR'] = df['TIME_BIN'].dt.strftime('%H:%M')
df['PASS'] = df['Result'].apply(lambda x: 1 if x == 'Pass' else 0)
df['ATTEMPTS_TRUNC'] = df['ATTEMPTS'].apply(lambda x: 4 if x > 4 else x)
df['FIRST_ATTEMPT'] = df['ATTEMPTS'].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
df.head()

In [73]:
#TC_ID, TestCount, Pass (percentage), AttemptNum, IsFirstAttempt, EmergencyStop, ReverseBayPark, ForwardBayPark, ParallelPark, ReverseRight
df_grouped = df.groupby(['TC_ID_y']).agg({'TC_ID_y': 'count', 'PASS': 'mean', 'ATTEMPTS':'mean', 'FIRST_ATTEMPT': 'mean', 'CONTROL_STOP_COMPLETED': 'mean', 'REVERSE_PARK_CARPARK': 'mean', 'TAXI_MANOEUVRE_COMPLETED': 'mean', 'REVERSE_PARK_ROAD': 'mean', 'REV_RIGHT_TRAIL_COMPLETED': 'mean'})
df_grouped.columns = ['totalTestCount', 'pass', 'attemptNum', 'isFirstAttempt', 'emergencyStop', 'reverseBayPark', 'forwardBayPark', 'parallelPark', 'reverseRight']
df_grouped['dailyTestCount'] = df_grouped['totalTestCount'] / time_span_days
df_grouped['anyManeuvre'] = df_grouped['reverseBayPark'] + df_grouped['forwardBayPark'] + df_grouped['parallelPark'] + df_grouped['reverseRight']

#Change TC_ID_y to TC_ID and cast to int
df_grouped.index.names = ['id']



In [75]:
#get Name from Driving Test Centres.csv
df_centres = pd.read_csv('Driving Test Centres.csv')
df_grouped_out = df_grouped.reset_index()
df_grouped_out = pd.merge(df_grouped_out, df_centres, on='id')
df_grouped_out = df_grouped_out.set_index('id')
df_grouped_out.head()

#reorder columns + ,latitude,longitude,addrLine1,addrLine2,addrLine3,addrLine4,addrLine5,postcode
df_grouped_out = df_grouped_out[['name', 'totalTestCount', 'dailyTestCount', 'pass', 'attemptNum', 'isFirstAttempt', 'emergencyStop', 'reverseBayPark', 'forwardBayPark', 'parallelPark', 'reverseRight', 'anyManeuvre', 'latitude', 'longitude', 'addrLine1', 'addrLine2', 'addrLine3', 'addrLine4', 'addrLine5', 'postcode']]

#to csv
df_grouped_out.to_csv('dtc_summary.csv', float_format='%.4f')